In [15]:
# Only once in a notebook
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_KEY")

# Create a single OpenAI client
client = OpenAI(api_key=openai_api_key)


In [ ]:
def func_a():
    return "Output from A"

def func_b():
    return "Output from B"

def func_c():
    return "Output from C"

def func_d():
    return "Output from D"

def func_e():
    return "Output from E"



In [32]:
class Agent:
    def __init__(self, goal, functions, client):
        self.goal = goal
        self.functions = functions
        self.client = client  # pass the already created client
        self.memory = []
        self.done = False
        self.action_ans = None  # store last chosen action

    def llm_decide(self, observation):
        if self.action_ans:
            # If an action was already decided, reuse it
            print("Calling in inbuilt action")
            print(self.action_ans)
            return self.action_ans
        else:
            # pass
            # Ask the LLM which function to run next
            prompt = f"""
You are an autonomous agent with a goal: {self.goal}.

Last observation: {observation}

Available actions: {list(self.functions.keys())} and FINISH.

Decide which action to take next. Respond ONLY with one of the action names.
"""
            response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
            )
            action = response.choices[0].message.content.strip()
            self.action_ans = action  # store in instance variable
            return action

    def run(self, max_steps=10):
        """
        Run the agent loop.
        max_steps limits the number of actions to avoid infinite loops
        """
        print(f"🚀 Agent started with goal: {self.goal}")
        observation = None
        steps = 0

        while not self.done and steps < max_steps:
            steps += 1
            action_name = self.llm_decide(observation)

            if action_name == "FINISH":
                self.done = True
                print("✅ Agent finished!")
                break

            func = self.functions.get(action_name)
            if func is None:
                print(f"⚠️ Invalid action from LLM: {action_name}. Skipping...")
                continue

            output = func()
            print(f"🧠 Agent chose {action_name}, output = {output}")
            self.memory.append((action_name, output))
            observation = output


In [33]:
# Create agent once
agent = Agent("Demo with remembered action", functions, client)

# First run: calls LLM
agent.run(max_steps=1)

# Second run: same action will be reused if self.action_ans is not reset
agent.run(max_steps=2)

# If you want to choose again, reset action_ans
# agent.action_ans = None
# agent.run(max_steps=2)


🚀 Agent started with goal: Demo with remembered action
🧠 Agent chose func_a, output = Output from A
🚀 Agent started with goal: Demo with remembered action
Calling in inbuilt action
func_a
🧠 Agent chose func_a, output = Output from A
Calling in inbuilt action
func_a
🧠 Agent chose func_a, output = Output from A


In [35]:
agent.memory

[('func_a', 'Output from A'),
 ('func_a', 'Output from A'),
 ('func_a', 'Output from A')]